In [1]:
from kipet.library.TemplateBuilder import *
from kipet.library.PyomoSimulator import *
from kipet.library.ParameterEstimator import *
from kipet.library.VarianceEstimator import *
from kipet.library.data_tools import *

import matplotlib.pyplot as plt
import numpy as np
import sys
import os

In [54]:
data = pd.DataFrame({'x': [1, 2, 3, 4], 'y':[2, 4, 6, 8]})
data.index = [0, 1, 2, 3]
data

,x,y
0,1,2
1,2,4
2,3,6
3,4,8


In [75]:
builder = TemplateBuilder()

In [76]:
builder.add_concentration_data(data)

   x  y
0  1  2
1  2  4
2  3  6
3  4  8


In [77]:
builder.add_parameter('b0', 0.0)
builder.add_parameter('b1', 0.0)

In [78]:
builder.add_mixture_component('dummy', 0.0)

In [79]:
builder.add_algebraic_variable('y')

In [80]:
def rule_odes(m, t):
    exprs = dict()
    exprs['dummy'] = 1
    return exprs
builder.set_odes_rule(rule_odes)

In [81]:
def rule_algebraics(m, t):
    r = list()
    r.append(m.Y[t, 'y'] - m.P['b0'] - m.P['b1'] * m.C[t, 'x'])
    return r
builder.set_algebraics_rule(rule_algebraics)

In [82]:
model = builder.create_pyomo_model(0, 3)

In [83]:
model.pprint()

19 Set Declarations
    C_indx : Dim=0, Dimen=2, Size=8, Domain=None, Ordered=Insertion, Bounds=None
        [(0, 'x'), (0, 'y'), (1, 'x'), (1, 'y'), (2, 'x'), (2, 'y'), (3, 'x'), (3, 'y')]
    S_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    U_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    X_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    Y_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    Z_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    algebraic_consts_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    algebraic_consts_index_1 : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=(0, 0)
        [0]
    algebraics : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=None
        ['y']
    allmeas_times : Dim=0, 

In [84]:
sigmas = {'y':1}

In [85]:
p_estimator = ParameterEstimator(model)
p_estimator.apply_discretization('dae.collocation',nfe=3,ncp=1,scheme='LAGRANGE-RADAU')

In [86]:
p_estimator.model.pprint()

19 Set Declarations
    C_indx : Dim=0, Dimen=2, Size=8, Domain=None, Ordered=Insertion, Bounds=None
        [(0, 'x'), (0, 'y'), (1, 'x'), (1, 'y'), (2, 'x'), (2, 'y'), (3, 'x'), (3, 'y')]
    S_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    U_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    X_index : Dim=0, Dimen=2, Size=0, Domain=None, Ordered=False, Bounds=None
        Virtual
    Y_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    Z_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    algebraic_consts_index : Dim=0, Dimen=2, Size=4, Domain=None, Ordered=False, Bounds=None
        Virtual
    algebraic_consts_index_1 : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=(0, 0)
        [0]
    algebraics : Dim=0, Dimen=1, Size=1, Domain=None, Ordered=False, Bounds=None
        ['y']
    allmeas_times : Dim=0, 

In [73]:
options = dict()
results_pyomo = p_estimator.run_opt('ipopt',
                                      variances=sigmas,
                                      tee=True,
                                      solver_opts = options)

Ipopt 3.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:       17
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       11
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       11
Total number of inequali

In [74]:
# And display the results
print("The estimated parameters are:")
for k,v in six.iteritems(results_pyomo.P):
    print(k, v)

The estimated parameters are:
b0 0.0
b1 0.0
